In [1]:
from pyscf import gto, scf, mcscf, fci
from pyscf.fci import cistring
import openfermion as of
import cirq
from openfermion.circuits import trotter
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt
import scipy
from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf
import cirq
import openfermion
from openfermion import InteractionOperator
from openfermion.circuits import trotter
from openfermion.transforms import get_interaction_operator
from cirq.testing import random_circuit


 done


  Preparing metadata (pyproject.toml) ... -

 done


  Using cached cirq_core-1.4.1-py3-none-any.whl.metadata (1.8 kB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl.metadata (4.6 kB)


  Using cached PubChemPy-1.0.4-py3-none-any.whl


...

 \

 |

 /

 done
  Created wheel for openfermion: filename=openfermion-1.7.0.dev0-py3-none-any.whl size=1209626 sha256=c465373d77b9e9b4e01d7044a26013e6b7ea3d2d367cc5f664078a49e3d96e6a
  Stored in directory: /tmpfs/tmp/pip-ephem-wheel-cache-1i0mk48_/wheels/7a/93/a6/5746203c4afd3b6e8d40a4edb587d608c51290286c5680f422


Successfully built openfermion


Double-factorized one-norm $\lambda(H_{DF})$ as a function of the squared free-fermions and eigenvalues up to rank $L = O(N)$

$$
\lambda(H_{DF}) = \sum_{ij} |h_{ij}^{(1)}| + \sum_{\ell=0}^{L-1} |\lambda_\ell| \left(\sum_{pq} |\left[g_{\ell}\right]_{pq}|\right)^2 .
$$

In [3]:

def create_controlled_trotter_circuit(hamiltonian: InteractionOperator, num_qubits: int, truncation_threshold: float = 1e-8, final_rank: int = None, spin_basis: bool = True):
    # Create the qubits for the system
    system_qubits = [cirq.NamedQubit(f'q{i}') for i in range(num_qubits)]
    
    # Add the control qubit
    control_qubit = cirq.NamedQubit('control')
    
    # Create the ControlledAsymmetricLowRankTrotterStep instance
    trotter_step = trotter.algorithms.low_rank.ControlledAsymmetricLowRankTrotterStep(
        hamiltonian=hamiltonian,
        truncation_threshold=truncation_threshold,
        final_rank=final_rank,
        spin_basis=spin_basis
    )
    
    return trotter_step


In [87]:
# Set Hamiltonian parameters for LiH simulation in active space.
diatomic_bond_length = 1.45
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
basis = 'sto-3g'
multiplicity = 1
active_space_start = 1
active_space_stop = 3

# Generate and populate instance of MolecularData.
molecule = of.MolecularData(geometry, basis, multiplicity, description="1.45")
molecule.load()

# Get the Hamiltonian in an active space.
hamiltonian = molecule.get_molecular_hamiltonian(
    occupied_indices=range(active_space_start),
    active_indices=range(active_space_start, active_space_stop))
# Create the controlled trotter circuit
n_qubits = of.count_qubits(hamiltonian)


controlled_trotter_circuit = create_controlled_trotter_circuit(hamiltonian,final_rank=2, num_qubits=n_qubits)

circ=cirq.Circuit(strategy=cirq.circuits.InsertStrategy.EARLIEST)

qubits = cirq.LineQubit.range(n_qubits+1)
# H gate


In [88]:
from cirq.testing import random_circuit

# Parameters
num_qubits = 4  # Number of qubits
n_moments = 10  # Number of moments (depth) of the random circuit
op_density = 0.7  # Operation density (probability of a gate appearing in a moment)

# Generate a random circuit
random_circ = random_circuit(qubits[1:], n_moments, op_density)
cirq.drop_negligible_operations(random_circ)


print(random_circ)

      ┌──┐
1: ─────@────H───────────────────X───────────×───
        │                        │           │
2: ────H┼────X───Y───────────────@───@───────┼───
        │    │                       │       │
3: ─────┼────@───────X───Y───────X───┼───────┼───
        │                        │   │       │
4: ─────@────────────────────Z───@───@───────×───
      └──┘


In [89]:
# Define a function to drop iSWAP gates
def drop_iswap_gates(circuit):
    new_circuit = cirq.Circuit()
    
    for moment in circuit:
        new_moment = []
        for op in moment:
            if not isinstance(op.gate, cirq.ISwapPowGate):
                new_moment.append(op)
        new_circuit.append(cirq.Moment(new_moment))
    
    return new_circuit
clifford_circuit=drop_iswap_gates(random_circ)
print(clifford_circuit)

      ┌──┐
1: ─────@────H───────────────────X───────────×───
        │                        │           │
2: ────H┼────X───Y───────────────@───@───────┼───
        │    │                       │       │
3: ─────┼────@───────X───Y───────X───┼───────┼───
        │                        │   │       │
4: ─────@────────────────────Z───@───@───────×───
      └──┘


In [90]:
def create_controlled_pauli_string_operator(pauli_tuple):
    # Map the tuple values to Cirq Pauli operators
    pauli_map = {0: cirq.I, 1: cirq.X, 2: cirq.Y, 3: cirq.Z}

    # Create the qubits
    qubits = [cirq.LineQubit(i+1) for i in range(len(pauli_tuple))]
    control_qubit = cirq.LineQubit(0)

    # Create the PauliString
    pauli_string = cirq.PauliString({qubits[i]: pauli_map[p] for i, p in enumerate(pauli_tuple) if p != 0})

    # Get the Pauli gates for the PauliString
    pauli_gates = [pauli_map[p] for p in pauli_tuple]

    # Create the controlled operation
    controlled_operations = []
    for qubit, gate in zip(qubits, pauli_gates):
        if gate != cirq.I:
            controlled_operations.append(cirq.ControlledGate(gate).on(control_qubit, qubit))

    # Create a circuit to demonstrate the controlled operation
    circuit = cirq.Circuit(controlled_operations)

    return circuit

# Example usage
pauli_tuple = (1, 3, 2, 2)  # Define your Pauli string tuple here
O_circ = create_controlled_pauli_string_operator(pauli_tuple)

# Print the circuit
print(O_circ)


0: ───@───@───@───@───
      │   │   │   │
1: ───X───┼───┼───┼───
          │   │   │
2: ───────Z───┼───┼───
              │   │
3: ───────────Y───┼───
                  │
4: ───────────────Y───


In [100]:
circ+=random_circ
circ+=cirq.H.on(qubits[0])

# CU(t, n_steps)
t1, t2 =1.2, 0.2
circ+=controlled_trotter_circuit.trotter_step(
    qubits=qubits[1:],
    time=t1,
    control_qubit=qubits[0])

circ+=cirq.X.on(qubits[0])
circ+=controlled_trotter_circuit.trotter_step(
    qubits=qubits[1:],
    time=t2,
    control_qubit=qubits[0])
circ+=cirq.X.on(qubits[0])

circ+=O_circ

imag=False
if imag:
    circ+=cirq.S(qubits[0])**-1
    
# H gate
circ+=cirq.H.on(qubits[0])

# Final measurement
#circ+=cirq.measure(qubits[0], key='anc.')

simulator = cirq.Simulator()
result = simulator.simulate(circ)


# Extract the state vector
final_state_vector = result.final_state_vector


# Separate the amplitudes for |0> and |1> on the ancilla qubit
amplitudes_0 = final_state_vector[:2]  # Amplitudes for ancilla |0>
amplitudes_1 = final_state_vector[2:]  # Amplitudes for ancilla |1>

# Compute probabilities
P0 = np.sum(np.abs(amplitudes_0)**2)
P1 = np.sum(np.abs(amplitudes_1)**2)

# Compute the expectation value
expectation_value = P0 - P1

print(f"P(0) = {P0}")
print(f"P(1) = {P1}")
print(f"Expectation Value (Re{{⟨ψ|O|ψ⟩}}): {expectation_value}")

P(0) = 0.08352338522672653
P(1) = 0.916475236415863
Expectation Value (Re{⟨ψ|O|ψ⟩}): -0.8329518437385559


In [96]:
simulator = cirq.Simulator()
result = simulator.simulate(cz_circuit)

# Separate the amplitudes for |0> and |1> on the ancilla qubit
amplitudes_0 = final_state_vector[:2]  # Amplitudes for ancilla |0>
amplitudes_1 = final_state_vector[2:]  # Amplitudes for ancilla |1>

# Compute probabilities
P0 = np.sum(np.abs(amplitudes_0)**2)
P1 = np.sum(np.abs(amplitudes_1)**2)

# Compute the expectation value
expectation_value = P0 - P1

print(f"P(0) = {P0}")
print(f"P(1) = {P1}")
print(f"Expectation Value (Re{{⟨ψ|O|ψ⟩}}): {expectation_value}")

P(0) = 0.08940756320953369
P(1) = 0.9105923175811768
Expectation Value (Re{⟨ψ|O|ψ⟩}): -0.8211847543716431


In [92]:
print(circ.to_text_diagram(transpose=True))

  0          1             2                    3                4
  │          │             │                    │                │
┌╴│          │             │                    │                │                   ╶┐
│ H          │             H                    │                │                    │
│ │          @─────────────┼────────────────────┼────────────────@                    │
└╴│          │             │                    │                │                   ╶┘
  │          │             │                    │                │
  │          H             X────────────────────@                │
  │          │             │                    │                │
  │          │             Y                    │                │
  │          │             │                    │                │
  │          │             │                    X                │
  │          │             │                    │                │
  │          │             │                 

In [93]:
# Print circuit.
cirq.drop_negligible_operations(circ)
gateset = cirq.CZTargetGateset(allow_partial_czs=False)
cz_circuit = cirq.optimize_for_target_gateset(circ, gateset=gateset)

print("Circuit compile time (to CZ gate set)")
depth = len(cirq.Circuit(cz_circuit.all_operations()))
print("Circuit Depth: "+str(depth))
print("-------------------------------------")
"""
    Get single and two qubit gates
"""
single_qubit_gates = 0
cz_gates = 0

for ops in cz_circuit.all_operations():
    
    if isinstance(ops.gate, cirq.PhasedXZGate):
        single_qubit_gates += 1
    
    else:
        cz_gates += 1
        
print("Single qubit gate count: " + str(single_qubit_gates))
print("CZ gate count: " + str(cz_gates))
print(" ")

Circuit compile time (to CZ gate set)
Circuit Depth: 563
-------------------------------------
Single qubit gate count: 812
CZ gate count: 336
 


In [95]:
print(cz_circuit.to_text_diagram(transpose=True))

  0                                      1                                      2                                    3                                 4
  │                                      │                                      │                                    │                                 │
  │                                      @──────────────────────────────────────┼────────────────────────────────────┼─────────────────────────────────@
  │                                      │                                      │                                    │                                 │
  │                                      PhXZ(a=0.5,x=-0.5,z=1)                 │                                    │                                 PhXZ(a=0,x=0,z=1)
  │                                      │                                      │                                    │                                 │
  │                                      │                        